In [1]:
import os
is_kaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE','')

In [2]:
from pathlib import Path
cred_path = Path('~/.kaggle/kaggle.json').expanduser()
if not cred_path.exists():
    creds = '{"username":"aspiringastronomer","key":"042f3b9ca8ccb28a6f804d983fa39a29"}'
    cred_path.parent.mkdir(exist_ok=True)
    cred_path.write_text(creds)
    cred_path.chmod(0o600)

In [3]:
path = Path('us-patent-phrase-to-phrase-matching') # us-patent-phrase-to-phrase-matching
if not is_kaggle and not path.exists():
    !pip install -Uqq kaggle
    import zipfile, kaggle
    
    kaggle.api.competition_download_cli(str(path))
    zipfile.ZipFile(f'{path}.zip').extractall(path)

In [4]:
!ls {path}

sample_submission.csv  test.csv  train.csv


In [5]:
import pandas as pd

In [6]:
df = pd.read_csv(path/'train.csv')
df.head()

,id,anchor,target,context,score
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75
2,36d72442aefd8232,abatement,active catalyst,A47,0.25
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00


In [7]:
df.describe(include='object')

,id,anchor,target,context
count,36473,36473,36473,36473
unique,36473,733,29340,106
top,37d61fd2272659b1,component composite coating,composition,H01
freq,1,152,24,2186


In [8]:
df['input'] = "TEXT1: " + df.context + "; TEXT2: " + df.target + "; ANC1: " + df.anchor
df.input.head()

0    TEXT1: A47; TEXT2: abatement of pollution; ANC...
1    TEXT1: A47; TEXT2: act of abating; ANC1: abate...
2    TEXT1: A47; TEXT2: active catalyst; ANC1: abat...
3    TEXT1: A47; TEXT2: eliminating process; ANC1: ...
4    TEXT1: A47; TEXT2: forest region; ANC1: abatement
Name: input, dtype: object

In [9]:
!pip install -Uqq datasets

In [10]:
from datasets import Dataset, DatasetDict
ds = Dataset.from_pandas(df)
ds

Dataset({
    features: ['id', 'anchor', 'target', 'context', 'score', 'input'],
    num_rows: 36473
})

In [11]:
#A deep learning model expects numbers as inputs, not English sentences! So we need to do two things:

# - Tokenization: Split each text up into words (or actually, as we'll see, into tokens)
# - Numericalization: Convert each word (or token) into a number.
model_nm = 'microsoft/deberta-v3-small'

In [12]:
from transformers import AutoModelForSequenceClassification,AutoTokenizer
tokz = AutoTokenizer.from_pretrained(model_nm)

Downloading:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/578 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.35M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.9/dist-packages/transformers/convert_slow_tokenizer.py:434: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [13]:
tokz.tokenize("Day 1 of 365 - Learn. Build. Iterate! Attention is all you need.")

['▁Day',
 '▁1',
 '▁of',
 '▁365',
 '▁-',
 '▁Learn',
 '.',
 '▁Build',
 '.',
 '▁It',
 'erate',
 '!',
 '▁Attention',
 '▁is',
 '▁all',
 '▁you',
 '▁need',
 '.']

In [14]:
tokz.tokenize("The blue whale (Balaenoptera musculus) is a marine mammal and a baleen whale. Reaching a maximum confirmed length of 29.9 meters and weighing up to 199 tonnes, it is the largest animal known to have ever existed")

['▁The',
 '▁blue',
 '▁whale',
 '▁(',
 'B',
 'ala',
 'en',
 'optera',
 '▁musc',
 'ulus',
 ')',
 '▁is',
 '▁a',
 '▁marine',
 '▁mammal',
 '▁and',
 '▁a',
 '▁bale',
 'en',
 '▁whale',
 '.',
 '▁Reaching',
 '▁a',
 '▁maximum',
 '▁confirmed',
 '▁length',
 '▁of',
 '▁29',
 '.',
 '9',
 '▁meters',
 '▁and',
 '▁weighing',
 '▁up',
 '▁to',
 '▁199',
 '▁tonnes',
 ',',
 '▁it',
 '▁is',
 '▁the',
 '▁largest',
 '▁animal',
 '▁known',
 '▁to',
 '▁have',
 '▁ever',
 '▁existed']

In [15]:
def tok_func(x): return tokz(x["input"])

In [16]:
tok_ds = ds.map(tok_func, batched=True)
# This adds a new item to our dataset called input_ids. For instance, here is the input and IDs for the first row of our data:

  0%|          | 0/37 [00:00<?, ?ba/s]

In [17]:
tok_ds = tok_ds.rename_columns({'score':'labels'})
rows = tok_ds[:1]
rows['input'], rows['input_ids']

(['TEXT1: A47; TEXT2: abatement of pollution; ANC1: abatement'],
 [[1,
   54453,
   435,
   294,
   336,
   5753,
   346,
   54453,
   445,
   294,
   47284,
   265,
   6435,
   346,
   23702,
   435,
   294,
   47284,
   2]])

In [18]:
eval_df = pd.read_csv(path/'test.csv')
eval_df.describe()

,id,anchor,target,context
count,36,36,36,36
unique,36,34,36,29
top,4112d61851461f60,el display,inorganic photoconductor drum,G02
freq,1,2,1,3


In [19]:
eval_df.head()

,id,anchor,target,context
0,4112d61851461f60,opc drum,inorganic photoconductor drum,G02
1,09e418c93a776564,adjust gas flow,altering gas flow,F23
2,36baf228038e314b,lower trunnion,lower locating,B60
3,1f37ead645e7f0c8,cap component,upper portion,D06
4,71a5b6ad068d531f,neural stimulation,artificial neural network,H04


In [20]:

dds = tok_ds.train_test_split(0.25, seed=42)
dds

DatasetDict({
    train: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 27354
    })
    test: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9119
    })
})

In [21]:
eval_df['input'] = 'TEXT1: ' + eval_df.context + '; TEXT2: ' + eval_df.target + '; ANC1: ' + eval_df.anchor
eval_ds = Dataset.from_pandas(eval_df).map(tok_func, batched=True)


  0%|          | 0/1 [00:00<?, ?ba/s]

In [22]:
eval_ds

Dataset({
    features: ['id', 'anchor', 'target', 'context', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 36
})

In [23]:
import numpy as np, matplotlib.pyplot as plt

def corr(x,y): return np.corrcoef(x,y)[0][1]

def show_corr(df, a, b):
    x,y = df[a],df[b]
    plt.scatter(x,y, alpha=0.5, s=4)
    plt.title(f'{a} vs {b}; r: {corr(x, y):.2f}')
    
def corr_d(eval_pred): return {'pearson': corr(*eval_pred)}

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [24]:
from transformers import TrainingArguments,Trainer

bs = 128
epochs = 4
lr = 8e-5


In [25]:
args = TrainingArguments('outputs', learning_rate=lr, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=True,
    evaluation_strategy="epoch", per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,
    num_train_epochs=epochs, weight_decay=0.01, report_to='none')

In [26]:
model = AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=1)
trainer = Trainer(model, args, train_dataset=dds['train'], eval_dataset=dds['test'],
                  tokenizer=tokz, compute_metrics=corr_d)


Downloading:   0%|          | 0.00/273M [00:00<?, ?B/s]

Some weights of the model checkpoint at microsoft/deberta-v3-small were not used when initializing DebertaV2ForSequenceClassification: ['mask_predictions.classifier.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.dense.bias', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.dense.weight', 'mask_predictions.dense.weight', 'mask_predictions.classifier.weight', 'mask_predictions.LayerNorm.weight']
- This IS expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from 

In [27]:
train_result = trainer.train()

The following columns in the training set don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: input, target, anchor, context, id. If input, target, anchor, context, id are not expected by `DebertaV2ForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 27354
  Num Epochs = 4
  Instantaneous batch size per device = 128
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 1
  Total optimization steps = 856


Epoch,Training Loss,Validation Loss,Pearson
1,No log,0.025698,0.799198
2,No log,0.025226,0.823025
3,0.031700,0.023134,0.830327
4,0.031700,0.022826,0.832513


The following columns in the evaluation set don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: input, target, anchor, context, id. If input, target, anchor, context, id are not expected by `DebertaV2ForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 9119
  Batch size = 256
The following columns in the evaluation set don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: input, target, anchor, context, id. If input, target, anchor, context, id are not expected by `DebertaV2ForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 9119
  Batch size = 256
Saving model checkpoint to outputs/checkpoint-500
Configuration saved in outputs/checkpoint-500/config.json
Model weights saved in outputs/checkpoint-500/pytorch_model.bin
tokenizer config file saved 

In [29]:
trainer.save_model(model_nm)

Saving model checkpoint to microsoft/deberta-v3-small
Configuration saved in microsoft/deberta-v3-small/config.json
Model weights saved in microsoft/deberta-v3-small/pytorch_model.bin
tokenizer config file saved in microsoft/deberta-v3-small/tokenizer_config.json
Special tokens file saved in microsoft/deberta-v3-small/special_tokens_map.json


In [30]:
preds = trainer.predict(eval_ds).predictions.astype(float)
preds

The following columns in the test set don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: input, target, anchor, context, id. If input, target, anchor, context, id are not expected by `DebertaV2ForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 36
  Batch size = 256


array([[ 0.50732422],
       [ 0.71484375],
       [ 0.55419922],
       [ 0.29467773],
       [-0.03997803],
       [ 0.49511719],
       [ 0.56640625],
       [-0.02009583],
       [ 0.24560547],
       [ 1.06640625],
       [ 0.26586914],
       [ 0.27563477],
       [ 0.78710938],
       [ 0.83642578],
       [ 0.75146484],
       [ 0.43457031],
       [ 0.25317383],
       [-0.01687622],
       [ 0.66455078],
       [ 0.34277344],
       [ 0.45410156],
       [ 0.21435547],
       [ 0.0670166 ],
       [ 0.23718262],
       [ 0.57470703],
       [-0.01470184],
       [-0.04037476],
       [-0.03207397],
       [-0.04214478],
       [ 0.63867188],
       [ 0.43017578],
       [ 0.046875  ],
       [ 0.73144531],
       [ 0.5234375 ],
       [ 0.47924805],
       [ 0.21081543]])

In [31]:
preds = np.clip(preds, 0, 1)

In [32]:
preds

array([[0.50732422],
       [0.71484375],
       [0.55419922],
       [0.29467773],
       [0.        ],
       [0.49511719],
       [0.56640625],
       [0.        ],
       [0.24560547],
       [1.        ],
       [0.26586914],
       [0.27563477],
       [0.78710938],
       [0.83642578],
       [0.75146484],
       [0.43457031],
       [0.25317383],
       [0.        ],
       [0.66455078],
       [0.34277344],
       [0.45410156],
       [0.21435547],
       [0.0670166 ],
       [0.23718262],
       [0.57470703],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.63867188],
       [0.43017578],
       [0.046875  ],
       [0.73144531],
       [0.5234375 ],
       [0.47924805],
       [0.21081543]])

In [33]:
import datasets
submission = datasets.Dataset.from_dict({
    'id': eval_ds['id'],
    'score': preds
})
submission.to_csv('submission.csv', index=False)

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

1021

In [34]:
eval_df.head()

,id,anchor,target,context,input
0,4112d61851461f60,opc drum,inorganic photoconductor drum,G02,TEXT1: G02; TEXT2: inorganic photoconductor dr...
1,09e418c93a776564,adjust gas flow,altering gas flow,F23,TEXT1: F23; TEXT2: altering gas flow; ANC1: ad...
2,36baf228038e314b,lower trunnion,lower locating,B60,TEXT1: B60; TEXT2: lower locating; ANC1: lower...
3,1f37ead645e7f0c8,cap component,upper portion,D06,TEXT1: D06; TEXT2: upper portion; ANC1: cap co...
4,71a5b6ad068d531f,neural stimulation,artificial neural network,H04,TEXT1: H04; TEXT2: artificial neural network; ...


In [50]:
valid_df = dds['test'].rename_columns({'labels':'score'}).to_pandas()[['id','anchor','target','context','score']]
valid_df.head()

,id,anchor,target,context,score
0,1d93fcef824082a3,protocol component,circuit,A46,0.25
1,9658a68dedd1b4cc,adjacent laterally,radius,A41,0.25
2,fdd44f80c6009fbd,free fatty acid,sore,C12,0.00
3,8445a426162619dc,hydrocarbyl substituted succinic,dicarboxylic,C10,0.50
4,eabf6e3ebaeae987,metatarsal bones,metatarsal,A61,0.50


In [51]:
valid_df['input'] = 'TEXT1: ' + valid_df.context + '; TEXT2: ' + valid_df.target + '; ANC1: ' + valid_df.anchor
valid_ds = Dataset.from_pandas(valid_df).map(tok_func, batched=True)
valid_ds[0]

  0%|          | 0/10 [00:00<?, ?ba/s]

{'id': '1d93fcef824082a3',
 'anchor': 'protocol component',
 'target': 'circuit',
 'context': 'A46',
 'score': 0.25,
 'input': 'TEXT1: A46; TEXT2: circuit; ANC1: protocol component',
 'input_ids': [1,
  54453,
  435,
  294,
  336,
  5718,
  346,
  54453,
  445,
  294,
  4823,
  346,
  23702,
  435,
  294,
  6624,
  3480,
  2],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [52]:
valid_preds = trainer.predict(valid_ds)
valid_preds

The following columns in the test set don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: input, target, anchor, context, score, id. If input, target, anchor, context, score, id are not expected by `DebertaV2ForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 9119
  Batch size = 256


PredictionOutput(predictions=array([[ 0.3193  ],
       [ 0.2573  ],
       [-0.010704],
       ...,
       [ 0.4575  ],
       [ 0.524   ],
       [ 0.473   ]], dtype=float16), label_ids=None, metrics={'test_runtime': 20.8631, 'test_samples_per_second': 437.087, 'test_steps_per_second': 1.726})

In [53]:
#
valid_pf = valid_preds.predictions.astype(float)
valid_pf = np.clip(valid_pf, 0, 1)
len(valid_pf)

9119

In [54]:
valid_df['predicted_score'] = valid_pf

In [55]:
valid_df.head()

,id,anchor,target,context,score,input,predicted_score
0,1d93fcef824082a3,protocol component,circuit,A46,0.25,TEXT1: A46; TEXT2: circuit; ANC1: protocol com...,0.319336
1,9658a68dedd1b4cc,adjacent laterally,radius,A41,0.25,TEXT1: A41; TEXT2: radius; ANC1: adjacent late...,0.257324
2,fdd44f80c6009fbd,free fatty acid,sore,C12,0.00,TEXT1: C12; TEXT2: sore; ANC1: free fatty acid,0.000000
3,8445a426162619dc,hydrocarbyl substituted succinic,dicarboxylic,C10,0.50,TEXT1: C10; TEXT2: dicarboxylic; ANC1: hydroca...,0.287354
4,eabf6e3ebaeae987,metatarsal bones,metatarsal,A61,0.50,TEXT1: A61; TEXT2: metatarsal; ANC1: metatarsa...,0.574707


In [56]:

from sklearn.metrics import mean_squared_error
from sklearn.metrics import explained_variance_score
MSE = mean_squared_error(y_true = valid_df['score'], y_pred = valid_df['predicted_score'])
MSE
MSE**(0.5)

0.14964146302443787